In [ ]:
from PIL import Image
from glob import glob
import os
import numpy as np
import warnings 
import time
import random

warnings.filterwarnings('ignore')

In [ ]:
CROPPED_DIR = 'data/cropped/'
SIMPLIFIED_DIR = 'data/simplified/'

In [ ]:
from keras.layers import *
from keras.activations import *
from keras.initializers import *
from keras.models import Model
from keras.optimizers import *

In [ ]:
def generator(z=(100,)):
    input_ = Input(z)
    
    # 4 X 4, 512
    x = Dense(4*4*512, activation=LeakyReLU(0.2))(input_)
    x = Reshape((4, 4, 512))(x)
    
    # 8 X 8, 512
    x = Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', kernel_initializer=TruncatedNormal(stddev=0.02))(x) # strides=(2, 2) 사진의 크기를 2배로 키움
    x = BatchNormalization(momentum=0.9, epsilon=0.00005)(x)
    x = LeakyReLU(0.2)(x)
    
    # 16 X 16, 256
    x = Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', kernel_initializer=TruncatedNormal(stddev=0.02))(x) # strides=(2, 2) 사진의 크기를 2배로 키움
    x = BatchNormalization(momentum=0.9, epsilon=0.00005)(x)
    x = LeakyReLU(0.2)(x)
    
    # 32 X 32, 128
    x = Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', kernel_initializer=TruncatedNormal(stddev=0.02))(x) # strides=(2, 2) 사진의 크기를 2배로 키움
    x = BatchNormalization(momentum=0.9, epsilon=0.00005)(x)
    x = LeakyReLU(0.2)(x)
    
    # 64 X 64, 64
    x = Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', kernel_initializer=TruncatedNormal(stddev=0.02))(x) # strides=(2, 2) 사진의 크기를 2배로 키움
    x = BatchNormalization(momentum=0.9, epsilon=0.00005)(x)
    x = LeakyReLU(0.2)(x)
    
    # 64 X 64, 3
    x = Conv2DTranspose(3, (5, 5), padding='same', kernel_initializer=TruncatedNormal(stddev=0.02), activation='tanh')(x)
    model = Model(input_, x)
    model.summary()
    
    return model

In [ ]:
generator = generator()

In [ ]:
def discriminator(input_shape=(64, 64, 3)):
    input_layer= Input(input_shape)
    
    x = Conv2D(64, (5, 5), padding='same', strides=(2, 2), kernel_initializer=TruncatedNormal(stddev=0.02))(input_layer)
    x = BatchNormalization(momentum=0.9, epsilon=0.00005)(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(128, (5, 5), padding='same', strides=(2, 2), kernel_initializer=TruncatedNormal(stddev=0.02))(x)
    x = BatchNormalization(momentum=0.9, epsilon=0.00005)(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(256, (5, 5), padding='same', strides=(2, 2), kernel_initializer=TruncatedNormal(stddev=0.02))(x)
    x = BatchNormalization(momentum=0.9, epsilon=0.00005)(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(512, (5, 5), padding='same', strides=(2, 2), kernel_initializer=TruncatedNormal(stddev=0.02))(x)
    x = BatchNormalization(momentum=0.9, epsilon=0.00005)(x)
    x = LeakyReLU(0.2)(x)
    
    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(input_layer, x)
    model.summary()
    return model

In [ ]:
discriminator = discriminator()

In [ ]:
discriminator.compile(loss='binary_crossentropy', metrics=['acc'], optimizer=Adam(lr=0.00004, beta_1=0.5))

## Gan 정의

In [ ]:
input_noise = Input(shape=(100, ))
x = generator(input_noise)
gan_output = discriminator(x)

In [ ]:
gan = Model(input_noise, gan_output)

In [ ]:
gan.summary()

In [ ]:
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0004, beta_1=0.5))

## outlier 제거

In [ ]:
exclude_image = ["9746","9731","9717","9684","9637","9641","9642","9584","9541","9535",
"9250","9251","9252","9043","8593","8584","8052","8051","8008","7957",
"7958""7761","7762","9510","9307","4848","4791","4785","4465","2709",
"7724","7715","7309","7064","7011","6961","6962","6963","6960","6949",
"6662","6496","6409","6411","6406","6407","6170","6171","6172","5617",
"4363","4232","4086","4047","3894","3889","3493","3393","3362","2780",
"2710","2707","2708","2711","2712","2309","2056","1943","1760","1743",
"1702","1281","1272","772","736","737","691","684","314","242","191"]

In [ ]:
Image.open(CROPPED_DIR + '9717.png')

In [ ]:
exclude_image = [x + '.png' for x in exclude_image]

## Input 데이터 생성

In [ ]:
input_images = np.asarray([np.asarray(Image.open(x).resize((64, 64))) for x in glob(CROPPED_DIR + '*') if x not in exclude_image])

In [ ]:
input_images[0].shape

In [ ]:
len(input_images)

In [ ]:
# shuffle
np.random.shuffle(input_images)

## Batch 생성

In [ ]:
# batch 생성
def get_batches(data):
    batches = []
    batch_size = 64
    for i in range(int(data.shape[0] // batch_size)):
        batch = data[i * batch_size: (i + 1) * batch_size]
        augmented_images = []
        for image in batch:
            image = Image.fromarray(image)
            # 확률적으로 Image 횡방향 flip
            if random.choice([True, False]):
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
            augmented_images.append(np.asarray(image))
        batch = np.asarray(augmented_images)
        normalized_batch = (batch / 127.5)-1
        batches.append(normalized_batch)
        
    return np.asarray(batches)

In [ ]:
epochs=50

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def show_samples(sample_images, name, epoch):
    figure, axes = plt.subplots(1, len(sample_images), figsize = (IMAGE_SIZE, IMAGE_SIZE))
    for index, axis in enumerate(axes):
        axis.axis('off')
        image_array = sample_images[index]
        axis.imshow(image_array)
        image = Image.fromarray(image_array)
        #image.save(name+"_"+str(epoch)+"_"+str(index)+".png") 
    #plt.savefig(name+"_"+str(epoch)+".png", bbox_inches='tight', pad_inches=0)
    plt.show()
    plt.close()

In [ ]:
def summarize_epoch(d_losses, g_losses , data_shape, epoch, duration, input_z):
    minibatch_size = int(data_shape[0]//BATCH_SIZE)
    print("Epoch {}/{}".format(epoch, EPOCHS),
          "\nDuration: {:.5f}".format(duration),
          "\nD Loss: {:.5f}".format(np.mean(d_losses[-minibatch_size:])),
          "\nG Loss: {:.5f}".format(np.mean(g_losses[-minibatch_size:])))
    fig, ax = plt.subplots()
    plt.plot(d_losses, label='Discriminator', alpha=0.6)
    plt.plot(g_losses, label='Generator', alpha=0.6)
    plt.title("Losses")
    plt.legend()
    plt.savefig(OUTPUT_DIR + "losses_" + str(epoch) + ".png")
    plt.show()
    plt.close()
    test(input_z, epoch)

In [ ]:
def test(input_z, epoch):
    samples = generator.predict(input_z[:SAMPLES_TO_SHOW])
    sample_images = [((sample + 1.0) * 127.5).astype(np.uint8) for sample in samples]
    show_samples(sample_images, OUTPUT_DIR + "samples", epoch)

In [ ]:
# Hyperparameters
IMAGE_SIZE = 64
NOISE_SIZE = 100
LR_D = 0.00004
LR_G = 0.0004
BATCH_SIZE = 64
EPOCHS = 50 # For better results increase this value
BETA1 = 0.5
WEIGHT_INIT_STDDEV = 0.02
EPSILON = 0.00005
SAMPLES_TO_SHOW = 8
OUTPUT_DIR = './output'

In [ ]:
d_losses = []
g_losses = []
cum_d_loss = 0
cum_g_loss = 0

for epoch in range(epochs):
    epoch += 1
    start_time = time.time()
    for batch_images in get_batches(input_images):
        
        noise_data = np.random.normal(0, 1, size=(64, 100))
        
        generated_images = generator.predict(noise_data)
        
        noise_prop = 0.05
        real_labels = np.zeros((64, 1)) + np.random.uniform(0.0, 0.1, size=(64, 1))
        flipped_index = np.random.choice(np.arange(len(real_labels)), size=int(noise_prop * len(real_labels)))
        real_labels[flipped_index] = 1 - real_labels[flipped_index]
        
        discriminator.trainable = True
        
        # Train discriminator on real data
        d_loss_real = discriminator.train_on_batch(batch_images, real_labels)

        # Prepare labels for generated data
        fake_labels = np.ones((BATCH_SIZE, 1)) - np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
        flipped_idx = np.random.choice(np.arange(len(fake_labels)), size=int(noise_prop*len(fake_labels)))
        fake_labels[flipped_idx] = 1 - fake_labels[flipped_idx]
        
        # Train discriminator on generated data
        d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        cum_d_loss += d_loss
        d_losses.append(d_loss[0])
        
        
        # Train generator
        discriminator.trainable = False
        noise_data = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_SIZE))
        g_loss = gan.train_on_batch(noise_data, np.zeros((BATCH_SIZE, 1)))
        cum_g_loss += g_loss
        g_losses.append(g_loss)
        
    if epoch > 0 and epoch % 20 == 0 :
        print("saving model")
        discriminator.save_weights("desc-simposon-model.h5-" + str(epoch))
        gan.save_weights("gan-simposon-model.h5-" + str(epoch))

    # Plot the progress
    summarize_epoch(d_losses, g_losses, input_images.shape, epoch, time.time()-start_time, noise_data)